In [11]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import os

import bwsketchnet as sn
from data_layer import DataLayer, load_pretrained_model

FLAGS = {}
FLAGS['data_path'] = 'dataset/dataset_with_order_info_256.mat'
FLAGS['model_path'] = 'dataset/model_with_order_info_256.mat'
FLAGS['batch_size'] = 100
FLAGS['logdir'] = 'models/bwn-model/'
# Instantiate the data layer to have easy access of the dataset
dataset = DataLayer(FLAGS['data_path'], batch_size=FLAGS['batch_size'])
# Load the pretrained models
pretrained = load_pretrained_model(FLAGS['model_path'] if FLAGS['model_path'] else None)

Loading the dataset...
Dataset loaded!
Pretrained Model Loaded!


In [12]:
with tf.Graph().as_default():
    images_placeholder = tf.placeholder(tf.float32, name='images_pl')
    labels_placeholder = tf.placeholder(tf.float32, name='labels_pl')
    dr_placeholder = tf.placeholder_with_default(1.0, shape=(), name='dr_pl')
    
    logits = sn.inference(images_placeholder, dr_placeholder, pretrained=pretrained)
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        latest_ckpt_file = tf.train.latest_checkpoint(os.path.join(FLAGS['logdir'], 'ckpt'))
        saver.restore(sess, latest_ckpt_file)
        
        images, labels = dataset.next_batch_train()
        rets = sess.run(logits, feed_dict={ images_placeholder: images, labels_placeholder: labels })

In [13]:
print np.argmax(rets, axis=1)
print labels

[  0   0   0   0 203   0   0   0 179   0   0   0   0   0   0   0   0   0
   0   0   0 186   0   0   0   0   0   0   0 187   0   0   0   0   0   0
   0   0   0   0   0   0  86   0   0   0   0   0   0   0   0   0   0   0
   1  36   1   1  41 224 226   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
